In [ ]:
pip install bs4

In [ ]:
pip install requests

In [ ]:
'''
Scrape Google News using bs4
'''
import time
from datetime import datetime, timedelta
from requests import get
# from contextlib import closing
from bs4 import BeautifulSoup
import requests
from lxml import html
import pandas as pd
import time
# import datetime as datetime
import random
# Define start and end dates
# Define start and end dates
from datetime import datetime, timedelta

start_date = datetime.strptime("2022-01-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2022-01-04", "%Y-%m-%d").date()

Ticker = 'Apple'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
to_merge = pd.DataFrame({'Date': [],'Ticker': [], 'Url':[], 'headline': [], 'source': [], 'snippet': []})
it = 0

while start_date <= end_date:
    it += 1
    time.sleep(abs(random.gauss(0,2)))
    Date = str(start_date).replace("-","")
    URL = 'https://www.google.com/search?q='+Ticker+'+'+Date+'&source=lnms&tbm=nws'
    print(URL)
    try:
        raw_html = requests.get(URL, headers=headers)
        soup = BeautifulSoup(raw_html.text, 'lxml')
        title_list = soup.find_all("a", class_="l lLrAF")
        date_list = soup.find_all("span", class_="f nsa fwzPFf")
        source_list = soup.find_all("span", class_="xQ82C e8fRJf")
        snippet_list = soup.find_all("div", class_="st")
        print(title_list)
    except Exception as e:
        print(e)
    # Parse elements
    title = []
    date = []
    source = []
    snippet = []
    url = []
    for elem in title_list:
        valid = str(elem).replace("<em>", "").replace("</em>", "")[19:-4]
        out1 = valid.split("=")[1][1:-6]
        out2 = valid.split("=")[-1].split('>')[-1]
        url.append(out1)
        title.append(out2)
    for elem in date_list:
        valid = str(elem)[27:-7]
        date.append(valid)
    for elem in source_list:
        valid = str(elem)[27:-7]
        source.append(valid)
    for elem in snippet_list:
        valid = str(elem).replace("<em>", "").replace("</em>", "")[16:-10]
        snippet.append(valid)
    if len(title) == len(date) and len(title) == len(url) and len(title) == len(source) and len(title) == len(snippet):
        ticker = ['AAPL']*len(title)
        news = pd.DataFrame({'Date':date,'Ticker': ticker,'Url': url, 'headline':title, 'source': source, 'snippet': snippet})
        to_merge = pd.concat([to_merge,news])
    
    # Move to next date
    start_date += timedelta(days=1)
    print(start_date)
    time.sleep(1)

to_merge.to_csv('csv_files/PYPL_news.csv')


# new york times

In [ ]:
'''
Scrape NY Times using NY Times API
'''
from nytimesarticle import articleAPI
import time
import csv
api = articleAPI('kZz45A7j3sanOFqYWO9FRFCcpk6QpOao')

def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    
    news = []

    try:
        for i in articles['response']['docs']:
            dic = {}
            dic['id'] = i['_id']
            #if i['abstract'] is not None:
            #    dic['abstract'] = i['abstract'].encode("utf8")
            dic['headline'] = i['headline']['main'].encode("utf8")
            #dic['desk'] = i['news_desk']
            dic['date'] = i['pub_date'][0:10] # cutting time of day.
            #dic['section'] = i['section_name']
            if i['snippet'] is not None:
                dic['snippet'] = i['snippet'].encode("utf8")
                dic['source'] = i['source']
                dic['type'] = i['type_of_material']
                dic['url'] = i['web_url']
                dic['word_count'] = i['word_count']
                # locations
                locations = []
                for x in range(0,len(i['keywords'])):
                    if 'glocations' in i['keywords'][x]['name']:
                        locations.append(i['keywords'][x]['value'])
                        dic['locations'] = locations
                        # subject
                        subjects = []
                        for x in range(0,len(i['keywords'])):
                            if 'subject' in i['keywords'][x]['name']:
                                subjects.append(i['keywords'][x]['value'])
                                dic['subjects'] = subjects
                                news.append(dic)
    except:
        pass
    return(news)

def get_articles(date,query):
    '''
    This function accepts a year in string format (e.g.'1980')
    and a query (e.g.'Amnesty International') and it will
    return a list of parsed articles (in dictionaries)
    for that year.
    '''
    all_articles = []

    for i in range(0,100): #NYT limits pager to first 100 pages. But rarely will you find over 100 pages of results anyway.
        try:
            
            articles = api.search(q = query,
                                  fq = {'source':['Reuters','AP', 'The New York Times']},
                                  begin_date = date + '0101',
                                  end_date = date + '1231',
                                  sort='oldest',
                                  news_desk = 'business',
                                  subject = 'business',
                                  glocations = 'U.S.',
                                  page = str(i))
            print("he;;p")
            
            articles = parse_articles(articles)

            time.sleep(1)
            all_articles = all_articles + articles
        except:

            pass
    return(all_articles)



Ticker = ['AAPL']
for i in range(2020,2022):
    try:
        # print ('Processing' + str(i) + '...')
        Ticker_year =  get_articles(str(i),'stock market activity')
        Ticker = Ticker + Ticker_year
        keys = Ticker[0].keys()
        print(Ticker_year)
        with open('csv_files/Market1.csv', 'w') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(Ticker)
    except :
        pass


# <b> GOOGLE NEWS </b>

In [ ]:
pip install GoogleNews


In [ ]:
from GoogleNews import GoogleNews

date = '05/03/2018'

# Create a GoogleNews instance
gn = GoogleNews()

# Set the search parameters
gn.search('AAPL apple')
gn.set_time_range(date, date)
# Get the results
results = gn.results()

# Extract the headlines and summaries
headlines = [result['title'] for result in results]
summaries = [result['desc'] for result in results]

# Print the headlines and summaries
for headline, summary in zip(headlines, summaries):
    print(headline)
    print(summary)

In [ ]:
from datetime import datetime, timedelta

date = '05/03/2023'
dt = datetime.strptime(date, '%d/%m/%Y')

i = 3
while i > 0:
    dt = dt + timedelta(days=1)
    date = dt.strftime('%d/%m/%Y')
    print(date)
    i = i - 1


In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import time
from GoogleNews import GoogleNews

tickers = ["AAPL"]
days = 3

# Set up GoogleNews object
googlenews = GoogleNews(start='01/04/2023',end='today')

# Set up empty DataFrame to store results
df = pd.DataFrame(columns=["company", "date", "headline", "summary"])

# Loop over each day, fetch news for each ticker, and append to DataFrame
date_str = '01/01/2022'
for i in range(days):
    date = datetime.strptime(date_str, '%m/%d/%Y')
    date_str = (date + timedelta(days=1)).strftime('%m/%d/%Y')
    print(f"Fetching news for {date_str}")
    
    for ticker in tickers:
        googlenews.search(ticker)
        news = googlenews.results(sort=True)
        for j in range(min(5, len(news))):
            headline = news[j]["title"]
            summary = news[j]["desc"]
            df = df.append({"company": ticker, "date": date_str, "headline": headline, "summary": summary}, ignore_index=True)
    
    # Append to CSV file
    df.to_csv("news.csv", mode="a", header=not bool(i), index=False)
    df = pd.DataFrame(columns=["company", "date", "headline", "summary"])
    
    # Wait for 1 second before making the next request
    time.sleep(1)


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [16]:
from datetime import datetime, timedelta
from GoogleNews import GoogleNews
import pandas as pd
import time

# Set the start and end dates
start_date = '2023-01-01'
end_date = '2023-01-05'

# Convert start and end dates to datetime objects
start = datetime.strptime(start_date, '%Y-%m-%d')
end = datetime.strptime(end_date, '%Y-%m-%d')

# Create an empty dataframe to store the results
df = pd.DataFrame(columns=['company', 'date', 'headline', 'summary'])

# Loop over each day in the date range
while start <= end:
    # Convert the current date to string format
    date_str = start.strftime('%Y-%m-%d')
    print(f"Fetching news for {date_str}")
    
    # Create a GoogleNews instance
    gn = GoogleNews()

    # Set the search parameters
    gn.search('AAPL apple')
    gn.set_time_range(date_str, date_str)
    
    # Get the results
    results = gn.results()
    
    # Extract the headlines and summaries
    headlines = [result['title'] for result in results]
    summaries = [result['desc'] for result in results]
    
    # Append the results to the dataframe
    for headline, summary in zip(headlines, summaries):
        df = df.append({"company": "AAPL", "date": date_str, "headline": headline, "summary": summary}, ignore_index=True)
    
    # Wait for 1 second to avoid being blocked by Google
    time.sleep(1)
    
    # Increment the date by 1 day
    start += timedelta(days=1)

# Save the results to a CSV file
df.to_csv('news.csv', mode='a', index=False, header=False)



Fetching news for 2023-01-01
Fetching news for 2023-01-02
Fetching news for 2023-01-03
Fetching news for 2023-01-04
Fetching news for 2023-01-05


### GOOGLE

## GOOGLE

# GOOGLE

In [27]:
import time
from datetime import datetime, timedelta
from requests import get
# from contextlib import closing
from bs4 import BeautifulSoup
import requests
from lxml import html
import pandas as pd
import time
# import datetime as datetime
import random
# Define start and end dates
# Define start and end dates
from datetime import datetime, timedelta
import bs4





2022-01-11
2022-01-12
2022-01-12
2022-01-13
2022-01-13
2022-01-14
2022-01-14
2022-01-15


In [34]:
start_date = datetime.strptime("2022-01-11", "%Y-%m-%d").date()
end_date = datetime.strptime("2022-01-14", "%Y-%m-%d").date()

while start_date <= end_date:
    # time.sleep(abs(random.gauss(0,2)))
    # Date = str(start_date).replace("-","")
    # Move to next date
    new_date = start_date + timedelta(days=1)
    print(new_date)
    start_date = new_date 
    print(start_date)
    time.sleep(1)

2022-01-12
2022-01-12
2022-01-13
2022-01-13
2022-01-14
2022-01-14
2022-01-15
2022-01-15


In [20]:
import requests
from bs4 import BeautifulSoup

# Set the ticker symbol and date
ticker = "AAPL"
date = "2023-01-01"
url = f"https://www.ask.com/web?q={ticker}+news&date={date}"
print(url)
# Send a GET request to Ask.com and get the HTML response
response = requests.get(url)
html = response.content
print(response)
# Use BeautifulSoup to parse the HTML and extract the news articles
soup = BeautifulSoup(html, "html.parser")
articles = soup.find_all("article")
print(html)
# Loop through the articles and print the headline and summary for each one
for article in articles:
    headline = article.find("h2", class_="PartialSearchResults-item-title").text
    summary = article.find("div", class_="PartialSearchResults-item-abstract").text
    print(f"Heading: {headline}")
    print(f"Summary: {summary}")


https://www.ask.com/web?q=AAPL+news&date=2023-01-01
<Response [403]>
b'<!DOCTYPE html><html lang=\'en\'><head><meta charset=\'utf-8\'><meta name=\'viewport\' content=\'width=device-width,initial-scale=1\'><meta name=\'description\' content=\'px-captcha\'><title>Access to this page has been denied</title></head><body id=\'px-block\'><div class=\'px-logo\'><img width=\'400px\' src=\'https://amg-fe-app-images.s3.amazonaws.com/human/amg-captcha-block.png\' alt=\'Ask Media Group\'></div><script>window._pxVid=\'\',window._pxUuid=\'af9e54dc-ea7f-11ed-a599-655570496b57\',window._pxAppId=\'PX8ZOFP9vf\',window._pxHostUrl=\'https://collector-PX8ZOFP9vf.perimeterx.net\',window._pxCustomLogo=\'\',window._pxJsClientSrc=\'//client.perimeterx.net/PX8ZOFP9vf/main.min.js\',window._pxFirstPartyEnabled=\'false\',window[\'_\'+window._pxAppId]={challenge:{context:{headerText:\'Are you a person or a robot?\',headerColor:\'#4E4E4E\',headerFontSize:\'16px\',messageText:\'<p>Please don\xe2\x80\x99t take this pe